<a href="https://colab.research.google.com/github/ProfAI/tutorials/blob/master/Reti%20Neurali%20per%20la%20Sentiment%20Analysis%20con%20Keras/imdb_sentiment_analysis_with_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Come creare una Rete Neurale per la Sentiment Analysis con Keras
La Sentiment Analysis è uno dei campi più popolari del Natural Language Processing, il suo scopo è quello di classificare documenti di testo, come post, commenti o recensioni, in base alla loro poralità, cioè in base al sentimento positivo o negativo espresso nel testo. Le principali tecniche per la Sentiment Analysis possono essere raggruppate in 2 categorie: lessicali e statistichel. Le ultime, grazie ai progressi esponenziali nel settore del machine learning e all'applicazione delle reti neurali artificiali, sono diventate estremamente popolari. In questo tutorial costruiremo una rete neurale artificiale per eseguire la sentiment analysis di recensioni di film usando Keras, una popolare libreria per il Deep Learning, che funziona al di sopra di altre librerie per il calcolo tensoriale come Tensorflow, CNTK o Theano, e permette di sviluppare diverse architetture di reti neurali artificiali con poche righe di codice.

## Prerequisiti
Per seguire questo tutorial ai bisogno di una certa familiarità con il Machine Learning, se non sai proprio nulla a riguardo [parti da qui](http://blog.profession.ai/cosa-e-machine-learning/) e poi dai uno sguardo [a questo](http://blog.profession.ai/deep-learning-svelato-ecco-come-funzionano-le-reti-neurali-artificiali/).
<br>
Al momento la versione preinstallata in Colaboratory ha un bug che la rende incompatibile con alcune funzioni di Keras, quindi eseguiamo il downgrade alla versione precedente.

In [3]:
!pip install numpy==1.16.1

## Step 1 - Procuriamoci il Dataset
Il dataset che utilizzeremo per addestrare la nostra rete neurale è l'IMDB Movie Review Dataset, che contiene 50.000  esempi di recensioni di film (25.000 per l'addestramento e 25.000 per il testing) correttamente etichettate come positive o negative. Possiamo importare il dataset dentro un'array numpy usando le API di Keras.

In [0]:
from keras.datasets import imdb 

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

Il parametro *num_words* ci serve per definire il numero massimo di parole più frequenti da considerare. Così facendo abbiamo ottenuto 4 array:
* *X_train* che contiene le features degli esempi per l'addestramento.
* *y_train* che contiene i target per l'addestramento, cioè un singolo valore 0 o 1 che indicano rispettivamente se la recensione è negativa o positiva.
* *X_test* che contiene le features degli esempi per l'addestramento.
* *y_test* che contiene i target il test.

Ogni riga degli array con le features corrisponde ad una recensione, le recensioni sono già state codificate in numeri, ognuna di esse è infatti una lista di numeri dove ogni numero corrisponde alla posizione della corrispondente parola all'interno del vocabolario dell'intero corpus di testo.

In [5]:
print(X_train[159]) # [1, 6, 675, 7, 300, 127, 24, 895, 8, 2623, 89, 753, 2279, 5, 2, 78, 14, 20, 9]

[1, 6, 675, 7, 300, 127, 24, 895, 8, 2623, 89, 753, 2279, 5, 2, 78, 14, 20, 9]


## Step 2 (opzionale) - Come decodificare una recensione

Nonostante non sia essenziale ai fini della costruzione del modello, può essere utile conoscere come decodificare una recensione per risalire al testo originale. Per farlo dobbiamo procurarci il vocabolario che mappa le parole agli indici ed utilizzarlo per costruire un dizionario con la relazione inversa, cioè che mappa gli indici alle parole.

In [6]:
word_index = imdb.get_word_index()
word_index = dict([(value, key) for (key, value) in word_index.items()])

1646592/1641221 [==============================] - 0s 0us/step


Ora possiamo usare il dizionario *reverse_word_index* per decodificare una recensione.

In [7]:
decoded_review = [word_index.get(i - 3, '?') for i in X_train[159]]
decoded_review = ' '.join(decoded_review)
print(decoded_review) # ? a rating of 1 does not begin to express how dull depressing and ? bad this movie is

? a rating of 1 does not begin to express how dull depressing and ? bad this movie is


Gli indici delle parole hanno un offset di 3 rispetto al vocabolario, per questo otteniamo la parola corrispondente facendo *i-3*,  utilizzando il metodo *join* delle stringhe uniamo la lista di parole ottenuta dalla list comprehensions dividendole con degli spazi, in modo da ottenere una stringa. Dato che abbiamo limitato il vocabolario a solamente le 5000 parole più frequenti, alcune parole presenti all'interno di una recensione potrebbero essere mancanti, al loro posto verrà inserito un punto interrogativo.

## Step 3 - Preprocessiamo i dati
Utilizziamo il One Hot Encoding per creare quelle che saranno le features del nostro modello. Il one hot encoding si esegue creando, per ogni recensione, un array di lunghezza pari al numero totale di parole presenti all'interno dell'intero corpus di testo e inserendo dei valori 1 agli indici corrispondenti alle parole presenti nella frase e dei valori 0 altrimenti.

In [0]:
import numpy as np

def onehot_encoding(data, size):
    onehot = np.zeros((len(data), size))
    for i, d in enumerate(data):
        onehot[i,d] = 1.
    return onehot
  
X_train = onehot_encoding(X_train, 5000) # len = (25000, 5000)
X_test = onehot_encoding(X_test, 5000) # len = (25000, 5000)

Il risultato di questa operazione è una matrice sparsa, cioè una matrice contenete per la maggior parte dei valori 0.

## Step 4 - Costruiamo la Rete Neurale
Una rete neurale artificiale è un modello di machine learning che riesce apprendere relazioni non lineari nei dati, anche molto complesse, ispirandosi al funzionamento del cervello animale. Diversi neuroni sono disposti su diversi strati in sequenza e i neuroni di strati successivi sono connessi ai neuroni degli strati precedenti tramite dei pesi. Il primo strato di una rete neurale prende in input le features, l'ultimo strato fornisce l'output della rete, mentre gli strati intermedi, chiamati anche strati nascosti, utilizzano le features provenienti dallo strato precedente per apprendere nuove features più significative per l'obiettivo della nostra rete. Nell'ambito del deep learning i neuroni vengono chiamati anche unità o nodi, noi adotteremo quest'ultimo termine.

La rete neurale che creeremo avrà la seguente architettura:

* Uno strato di input con 512 nodi, cioè il numero di features del dataset.
* Uno strato nascosto con 128 nodi.
* Uno strato nascosto con 32 nodi.
* Uno strato nascosto con 8 nodi.
* Un'ultimo strato di output, con un solo nodo che conterrà l'output della rete, cioè il risultato della classificazione.

In [9]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(5000,)))
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

W0713 15:10:18.701968 139770219632512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0713 15:10:18.746067 139770219632512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0713 15:10:18.752196 139770219632512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



La classe Sequential ci permette di inizializzare un nuovo stack lineare di strati, utilizzando il metodo .add possiamo aggiungere nuovi strati. La classe Dense ci permette di creare un nuovo strato denso, cioè uno strato in cui tutti i nodi dello strato precedente sono connessi a tutti i nodi dello strato successivo tramite dei pesi. Per questa classe dobbiamo specificare dei parametri:
* il primo parametro è il numero di nodi dello strato, per l'ultimo strato è pari al numero di output, nel nostro caso, trattandosi di un problema di classificazione binaria (recensione positiva/negativa) avremo un unico nodo di output che conterrà la probabilità che la recensione sia positiva o negativa.
* il parametro *activation* è la funzione di attivazione da utilizzare per lo strato, quella che ci permette di ottenere risultati non lineari, la funzione di attivazione più utilizzata per gli strati nascosti è la Rectified Linear Unit (RELU), mentre per lo strato di output, trattandosi di un problema di classificazione binaria, dobbiamo utilizzare la Sigmoide.
* il parametro *input_shape* contiene la dimensione dell'input, va specificato solo per lo strato di input, mentre per gli altri strati viene calcolato in automatico da Keras.

Dopo aver definito l'architettura della rete dobbiamo configurare la fase di addestramento utilizzando il metodo *compile*.

In [10]:
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

W0713 15:10:21.907979 139770219632512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0713 15:10:21.937448 139770219632512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0713 15:10:21.943973 139770219632512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


* All'interno del parametro *loss* specifichiamo la funzione di costo da utilizzare, cioè la funzione che ci permette di calcolare la performance del nostro modello in base alla qualità delle sue predizioni. Nel caso di classificazioni binarie la funzione di costo da utilizzare è la *binary crossentropy*, che tiene conto della probabilità che il nostro modello abbiamo fornito il risultato corretto.
* All'interno del parametro *optimizer* specifichiamo l'algoritmo di ottimizzazione da utilizzare per l'addestramento del modello. Un'algoritmo di ottimizzazione ci permette di trovare i pesi del modello che minimizzano la funzione di costo da noi specificata, l'algoritmo di ottimizzazione principale è il *Gradient Descent*, che utilizza un processo iterativo in cui, ad ogni iterazione, ogni peso viene sommata alla relativa derivata rispetto alla funzione di costo moltiplicata per un'ulteriore iperparametro chiamato *learning rate*, utilizzato per controlare la dimensione di ogni step dell' ottimizzazione. Nel nostro caso utilizzeremo *adamax*, una variante del gradient descent che dovrebbe permettere di ottenere risultati migliori nel caso in cui le features siano rappresentate da una matrice sparsa.
* All'interno del parametro *metrics* specifichiamo una lista con altre metriche aggiuntive che ci permetteranno di misurare la qualità del modello, come l'*accuracy* che non è altro che la percentuale di classificazioni eseguite correttamente dal modello.

Adesso siamo pronti per avviare la fase di addestramento utilizzando il metodo fit, alla quale dobbiamo passare gli array con i dati per l'addestramento, features e target, il numero di epoche, ovvero di iterazioni dell'algoritmo di ottimizzazione, è la dimensione di ogni batch di addestramento, cioè il numero di esempi che verranno utilizzati per uno step dell'algoritmo di ottimizzazione.

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=512)

W0713 15:10:24.531879 139770219632512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
25000/25000 [==============================] - 5s 197us/step - loss: 0.3683 - acc: 0.8414
Epoch 2/10
25000/25000 [==============================] - 1s 42us/step - loss: 0.1997 - acc: 0.9248
Epoch 3/10
25000/25000 [==============================] - 1s 41us/step - loss: 0.1289 - acc: 0.9556
Epoch 4/10
25000/25000 [==============================] - 1s 40us/step - loss: 0.0594 - acc: 0.9854
Epoch 5/10
25000/25000 [==============================] - 1s 39us/step - loss: 0.0209 - acc: 0.9963
Epoch 6/10
25000/25000 [==============================] - 1s 38us/step - loss: 0.0069 - acc: 0.9989
Epoch 7/10
25000/25000 [==============================] - 1s 38us/step - loss: 0.0024 - acc: 0.9998
Epoch 8/10
25000/25000 [==============================] - 1s 38us/step - loss: 0.0011 - acc: 1.0000
Epoch 9/10
25000/25000 [==============================] - 1s 38us/step - loss: 5.8919e-04 - acc: 1.0000
Epoch 10/10
25000/25000 [==============================] - 1s 38us/step - loss: 4.0894e-04 - ac

Alla decima epoca abbiamo ottenuto un'accuracy del 100% ed un valore per la funzione di costo tendente allo zero, questo vuol dire che il nostro modello ha classificato correttamente tutti gli esempi del set di addestramento con un grado di incertezza bassissimo. Vediamo come se la cava con recensioni che non ha già visto durante l'addestramento usando il metodo *.evaluate* sul set di test.

In [12]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 2s 62us/step


[0.7001907266867161, 0.87088]

L'accuracy sul set di test è decisamente più bassa, mentre il valore della binary cross entropy è disastroso, come mai ? Perché il modello che abbiamo costruito ha memorizzati i dati di addestamento piuttosto che apprendere da essi e quindi ora fallisce nel generalizzare su nuovi dati. Questa condizione è conosciuta come overfitting ed è uno dei problemi principali del Machine Learning. Vediamo come possiamo risolverlo.

## Step 5 - Contrastiamo l'overfitting
Le soluzioni migliori per contrastare l'overfitting consistono nel ridurre la complessità del modello oppure nel raccogliere un numero maggiore di esempi per l'addestramento. Quando non è possibile far ciò possiamo adoperare delle tecniche di regolarizzazione, come:
* le regolarizzazioni L1 ed L2: che ci permettono di penalizzare i pesi eccessivamente grandi, che sono proprio quelli che causano l'overfitting.
* il dropout: che ci permette di "spegnere" una percentuale prefissata di nodi selezionati a caso, in questo modo i nodi eviteranno di farsi carico degli errori di altri nodi riducendo il rischio di overfitting.

Ridefiniamo l'architettura della rete aggiungendo la regolarizzazione L2 e il dropout tra gli strati.

In [0]:
from keras.regularizers import l2
from keras.layers import Dropout


model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(5000,), kernel_regularizer=l2(0.1)))
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(32,activation='relu',kernel_regularizer=l2(0.001)))
model.add(Dropout(0.4))
model.add(Dense(8,activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

La classe *Dropout* ha bisogno di un unico parametro, che rappresenta la percentuale di nodi da disattivare ad ogni iterazione. Anche la funzione *l2* necessita di un'unico parametro, che rappresenta il parametro di regolarizzazione, un valore che indica l'intensità della regolarizzazione da applicare. Fatto questo, riconfiguriamo la fase di addestramento e avviamola nuovamente, questa volta per 100 epoche.

In [31]:
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=512)

Epoch 1/100
25000/25000 [==============================] - 2s 72us/step - loss: 12.3156 - acc: 0.5902
Epoch 2/100
25000/25000 [==============================] - 1s 42us/step - loss: 0.8628 - acc: 0.7631
Epoch 3/100
25000/25000 [==============================] - 1s 41us/step - loss: 0.7241 - acc: 0.8240
Epoch 4/100
25000/25000 [==============================] - 1s 41us/step - loss: 0.6826 - acc: 0.8359
Epoch 5/100
25000/25000 [==============================] - 1s 41us/step - loss: 0.6451 - acc: 0.8438
Epoch 6/100
25000/25000 [==============================] - 1s 40us/step - loss: 0.6166 - acc: 0.8529
Epoch 7/100
25000/25000 [==============================] - 1s 41us/step - loss: 0.6155 - acc: 0.8544
Epoch 8/100
25000/25000 [==============================] - 1s 40us/step - loss: 0.5849 - acc: 0.8578
Epoch 9/100
25000/25000 [==============================] - 1s 39us/step - loss: 0.5802 - acc: 0.8566
Epoch 10/100
25000/25000 [==============================] - 1s 41us/step - loss: 0.5480 - 

I valori della funzione di costo e dell'accuracy sono più realistici rispetto a prima, verifichiamo se abbiamo risolto il nostro problema di overfitting testando il modello sul set di test.

In [0]:
 model.evaluate(X_test, y_test)

25000/25000 [==============================] - 3s 100us/step


[0.4818225428390503, 0.87684]

Le metriche sul set di test sono migliori rispetto a prima e più vicine a quelle ottenute sul set di addestramento, specialmente la binary cross entropy, questo sta ad indicare che il modello che abbiamo addestrato è più sicuro sulle sue predizioni.

## Step 6 - Mettiamo la Rete Neurale all'opera
Ora che la nostra rete neurale funziona abbastanza bene, mettiamola alla prova su nuove recensioni, cominciamo definendo una funzione che prende in input una recensione e la converte in un array numpy codificato tramite one hot encoding, pronto per essere dato in pasto alla nostra rete neurale.

In [0]:
from re import sub

def preprocess(review):
  
    # otteniamo il vocabolario
    word_index = imdb.get_word_index()
    
    # Rimuoviamo un'eventuale punteggiatura utilizzando un'espressione regolare
    review = sub(r'[^\w\s]','',review) 
    # Convertiamo tutto in minuscolo
    review = review.lower()
    # Creiamo un array di parole
    review = review.split(" ")

    # Qui dentro metteremo gli IDs
    review_array = []

    # Iteriamo lungo le parole della recensione
    for word in review:
        # proseguiamo se la parola si trova all'interno
        # della lista di parole del corpus di addestramento
        if word in word_index:
            # estraiamo l'indice della parola 
            index = word_index[word] 
            # Proseguiamo se l'indice è minore o uguale a 5000
            # cioè il numero di parole che abbiamo utilizzato
            # per l'addestramento
            if index <= 5000:
                # aggiungiamo l'indice all'array
                # ricordandoci dell'offset di 3
                review_array.append(word_index[word]+3)
                
    # Eseguiamo il one hot encoding sulla lista di indici
    print(review_array)
    review_array = onehot_encoding([review_array], 5000)
    return review_array


L'ouptut della nostra rete neurale sarà un valore compreso tra 0 ed 1 che indica la probabilità che la recensione sia positiva, quindi un output di 0 indicherà una recensione sicuramente non positiva (e quindi negativa), mentre un valore di 1 indicherà una recensione sicuramente positiva. Definiamo una funzione che prendendo in input questo valore ritorna una stringa che ne rappresenta il sentiment associato

In [0]:
def prob_to_sentiment(prob):
    
    if(prob>0.9): return "fantastica"
    elif(prob>0.75): return "ottima"
    elif(prob>0.55): return "buona" 
    elif(prob>0.45): return "neutrale"
    elif(prob>0.25): return "negativa"
    elif(prob>0.1): return "brutta"
    else: return "pessima"


Ora mettiamo tutto insieme per classifcare qualche recensione pescata dal web, cominciamo con una relativa a uno dei film più brutti che ho avuto la sciagura di vedere: Paranormal Activity 4.

In [34]:
review = "what a waste of time and cash.. the movie was pointless. with no flow. no questions answered. just a waste. I never review movies but had to share how bad this was..compared to part 1- 2- and 3.... i don't know what else to say other than how misleading the commercial is.. the commercial was cut and spliced with video and audio that didn't even match what happened in the movie... you have been warned. when the movie was over.. people actually Boo'd. hopefully people will spread the word, and save others from throwing their money away. i know die-hard fans will go and give it a shot, but will be disappointed as well. Sinister was better and actually made you jump quite a few times."
x = preprocess(review)
y = model.predict(x)[0]
print("REVIEW:", review)
print("\n")
print("La recensione è %s [%.6f]" % (prob_to_sentiment(y), y))

[51, 6, 437, 7, 58, 5, 2208, 4, 20, 16, 1149, 19, 57, 2973, 57, 1204, 43, 6, 437, 13, 115, 733, 102, 21, 69, 8, 1497, 89, 78, 14, 8, 173, 300, 241, 5, 342, 13, 124, 51, 334, 8, 135, 85, 74, 89, 4, 2153, 9, 4, 2153, 16, 605, 5, 19, 374, 5, 3884, 15, 60, 1014, 51, 575, 11, 4, 20, 25, 28, 77, 2815, 54, 4, 20, 16, 120, 84, 165, 2363, 84, 80, 4600, 4, 681, 5, 607, 409, 39, 2825, 68, 278, 245, 13, 124, 451, 80, 140, 5, 202, 12, 6, 324, 21, 80, 30, 685, 17, 73, 2950, 16, 128, 5, 165, 93, 25, 1783, 179, 6, 171, 211]
REVIEW: what a waste of time and cash.. the movie was pointless. with no flow. no questions answered. just a waste. I never review movies but had to share how bad this was..compared to part 1- 2- and 3.... i don't know what else to say other than how misleading the commercial is.. the commercial was cut and spliced with video and audio that didn't even match what happened in the movie... you have been warned. when the movie was over.. people actually Boo'd. hopefully people will sp

La nostra rete indica che la recensione è (ovviamente) pessima, ma proprio tanto tanto. Proviamo adesso con una recensione che riguarda Avengers: Infinity War.



In [29]:
review = "This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be."
x = preprocess(review)
y = model.predict(x)

print("REVIEW:", review)
print("\n")
print("La recensione è %s [%.6f]" % (prob_to_sentiment(y), y))

[14, 20, 80, 2479, 129, 330, 5, 989, 129, 483, 5, 97, 25, 1680, 8, 140, 145, 18, 53, 2510, 530, 5, 128, 74, 12, 47, 101, 208, 8, 30]
REVIEW: This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be.


La recesione è fantastica [0.915910]


La rete dice che la recensione è positiva (dai, a chi non è piaciuto questo film ?). Se vuoi divertirti un po' prova a scrivere la tua recensione, tenendo conto che, dato che abbiamo addestrato la rete su di un corpus di testo inglese, la recensione deve essere in lingua inglese.

In [0]:
review = input("Write your review: ")
x = preprocess(review)
prob = model.predict(review)

print("REVIEW: %s" % review)
print("\n")
print("La recensione è %s [%.6f]" % (prob_to_sentiment(prob), prob))